### 1. Download the dataset MM-IMDB from https://www.kaggle.com/johnarevalo/datasets. Select 2000 images with drama label and 2000 with horror label. Be careful to exclude those images with both labels. Perform a 60%-20%-20% stratified split for training, validation and test datasets.
### 2. Use a fine tuning approach to train a ResNet50 network to discriminate between drama and horror movies.
### 3. Evaluate the model over the test dataset. Discuss the results.